### Topic model

In [1]:
### Load libraries -------
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import os
from datetime import datetime, date, timedelta
import numpy as np
import io
import requests
from PyPDF2 import PdfFileReader
from pdfminer.high_level import extract_text

import nltk
from nltk.tokenize import word_tokenize
#nltk.download('punkt')
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
import re

# Pandas parameters
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_rows', 2000)
#pd.set_option('display.max_columns', 500)

In [2]:
### Define out paths ------

# check if out path exists, if it does not exists add the path
if not os.path.exists('/Users/hannahfrank/desktop/qca_peace_agreements_out'):
    os.makedirs('/Users/hannahfrank/desktop/qca_peace_agreements_out')

# add subfolders for the out path
out_paths = {
    "desciptive_plots_outcome": '/Users/hannahfrank/desktop/qca_peace_agreements_out/desciptive_plots_outcome',
    "data": '/Users/hannahfrank/desktop/qca_peace_agreements_out/data'
}

for key, val in out_paths.items():
    if not os.path.exists(val):
        os.makedirs(val)
        
out_paths["data"]

'/Users/hannahfrank/desktop/qca_peace_agreements_out/data'

In [3]:
### Load data from local folder ----------
ucdp = pd.read_csv(os.path.join(out_paths["data"], 
                                "ucdp_full.csv"))
ucdp.head(3)

,paid,gwno,dyad_id,dyad_name,pa_name,year,pa_date,pa_comment,pa_sign,c_sign,...,gender,active_conflict,termdur,txt,linktofulltextagreement,inclusive,out_iss,frame,fulltext,ready
0,1565,700,"729, 726, 727, 725, 730, 732",Government of Afghanistan - Mahaz-i Milli-yi I...,Peshawar Accord,1992,1992-04-24,Partial Peace Agreement \n\nThis agreement set...,"No signatures, but agreement mentions: Jamiat-...","Dyadic peace agreement signed in Peshawar, Pak...",...,0,1,-99,1,http://ucdpged.uu.se/peaceagreements/fulltext/...,2,2,1,"[['Peshawar Accord', ' ', 'In the Name of Alla...",1
1,1128,700,"726, 732",Government of Afghanistan - Hizb-i Islami-yi A...,Afghan Peace Accord and Annex on the Division ...,1993,1993-03-07,Partial peace agreement \n\nA number of war...,Professor Burhan-ud-Din Rabbani of Jamiat-i-Is...,Dyadic peace agreement\nThe negotiations prior...,...,0,1,-99,1,http://ucdpged.uu.se/peaceagreements/fulltext/...,2,3,2,[['Afghan Peace Accord (Islamabad Accord) Giv...,1
2,1488,700,726,Government of Afghanistan - Hizb-i Islami-yi A...,Kabul Agreement,2016,2016-09-22,Full peace agreement\n\nAfter periods of negot...,For the Government of Afghanistan: President M...,Dyadic agreement signed in Kabul on 22 Septemb...,...,1,1,-99,1,http://ucdpged.uu.se/peaceagreements/fulltext/...,2,0,2,[['Kabul Agreement between the Government of A...,1


In [4]:
### Only ready documets ----
ucdp_full = ucdp.loc[ucdp["ready"]==1].copy()
ucdp_full = ucdp_full.reset_index(drop=True).copy()
ucdp_full

,paid,gwno,dyad_id,dyad_name,pa_name,year,pa_date,pa_comment,pa_sign,c_sign,...,gender,active_conflict,termdur,txt,linktofulltextagreement,inclusive,out_iss,frame,fulltext,ready
0,1565,700,"729, 726, 727, 725, 730, 732",Government of Afghanistan - Mahaz-i Milli-yi I...,Peshawar Accord,1992,1992-04-24,Partial Peace Agreement \n\nThis agreement set...,"No signatures, but agreement mentions: Jamiat-...","Dyadic peace agreement signed in Peshawar, Pak...",...,0,1,-99,1,http://ucdpged.uu.se/peaceagreements/fulltext/...,2,2,1,"[['Peshawar Accord', ' ', 'In the Name of Alla...",1
1,1128,700,"726, 732",Government of Afghanistan - Hizb-i Islami-yi A...,Afghan Peace Accord and Annex on the Division ...,1993,1993-03-07,Partial peace agreement \n\nA number of war...,Professor Burhan-ud-Din Rabbani of Jamiat-i-Is...,Dyadic peace agreement\nThe negotiations prior...,...,0,1,-99,1,http://ucdpged.uu.se/peaceagreements/fulltext/...,2,3,2,[['Afghan Peace Accord (Islamabad Accord) Giv...,1
2,1488,700,726,Government of Afghanistan - Hizb-i Islami-yi A...,Kabul Agreement,2016,2016-09-22,Full peace agreement\n\nAfter periods of negot...,For the Government of Afghanistan: President M...,Dyadic agreement signed in Kabul on 22 Septemb...,...,1,1,-99,1,http://ucdpged.uu.se/peaceagreements/fulltext/...,2,0,2,[['Kabul Agreement between the Government of A...,1
3,1021,540,714,Government of Angola - UNITA,The Gbadolite declaration,1989,1989-06-22,Peace process agreement\n\nThe Gbadolite Decla...,President Jose Eduardo dos Santos signed for t...,Comprehensive peace agreement\nThe Gbadolite D...,...,0,1,-99,1,http://ucdpged.uu.se/peaceagreements/fulltext/...,1,4,1,[[' The Gbadolite Declaration on Angola 1) At...,1
4,1022,540,714,Government of Angola - UNITA,The Bicesse Agreement,1991,1991-05-31,Peace process Agreement\n\nThe Bicesse Accords...,President Jose Eduardo dos Santos signed for t...,Comprehensive peace agreement\nThe Bicesse Acc...,...,0,1,-99,1,http://ucdpged.uu.se/peaceagreements/fulltext/...,1,2,1,"[[' The Bicesse Agreement ', 'The Government o...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,1368,500,688,Government of Uganda - LRA,Agreement on Accountability and Reconciliation...,2007,2007-06-29,Partial peace agreement\n\nThe Agreement on Ac...,"For the Government of Uganda: Dr S Kagoda, Per...",NaN,...,1,1,-99,1,http://ucdpged.uu.se/peaceagreements/fulltext/...,2,0,1,[[' AGREEMENT ON ACCOUNTABILITY AND RECONCILIA...,1
232,1377,500,688,Government of Uganda - LRA,Annex to the accountability and reconciliation...,2008,2008-02-19,Partial peace agreement\n\nThe annex provides ...,Hon. Ruhakana Rugtinefa (Dr) Minister of Inter...,Dyadic peace agreement,...,1,1,-99,1,http://ucdpged.uu.se/peaceagreements/fulltext/...,2,0,1,"[[""Annexure to the Agreement on Accountability...",1
233,1063,200,692,Government of United Kingdom - PIRA,The Good Friday Agreement,1998,1998-04-10,Full peace agreement\n\nAfter a long series of...,The British-Irish Agreement was signed by: Ton...,Dyadic peace agreement\nSinn Féin said they ne...,...,1,1,-99,1,http://ucdpged.uu.se/peaceagreements/fulltext/...,2,0,2,[['AGREEMENT REACHED IN THE MULTI-PARTY NEGOTI...,1
234,1535,678,10855,Government of Yemen (North Yemen) - Ansarallah,The Peace and National Partnership Agreement,2014,2014-09-21,Partial Peace Agreement\n\nThis agreement prov...,"Not signed, but agreement mentions the followi...",Dyadic peace agreement signed on 21 September ...,...,1,1,-99,1,http://ucdpged.uu.se/peaceagreements/fulltext/...,2,4,1,[['The Peace and National Partnership Agreemen...,1


In [5]:
#https://blog.ekbana.com/pre-processing-text-in-python-ad13ea544dae

def to_lower(text):
    return ' '.join([w.lower() for w in word_tokenize(text)])

def remove_num(text):
    return ''.join(c for c in text if not c.isdigit())


In [12]:
ucdp_full["tokens"][0]=word_tokenize(ucdp_full["fulltext"][0])
ucdp_full["tokens"][0]=re.sub('<[^<]+?>','', ucdp_full["fulltext"][0])
ucdp_full["tokens"][0]=to_lower(ucdp_full["fulltext"][0])
ucdp_full["tokens"][0]=remove_num(ucdp_full["fulltext"][0])
ucdp_full["tokens"][0]=tokenizer.tokenize(ucdp_full["fulltext"][0])

<ipython-input-12-f3937e111b44>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ucdp_full["tokens"][0]=word_tokenize(ucdp_full["fulltext"][0])
<ipython-input-12-f3937e111b44>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ucdp_full["tokens"][0]=re.sub('<[^<]+?>','', ucdp_full["fulltext"][0])
<ipython-input-12-f3937e111b44>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ucdp_full["tokens"][0]=to_lower(ucdp_full["fulltext"][0])
<ipy

In [6]:
ucdp_full["tokens"]=""
tokenizer = RegexpTokenizer(r'\w+')

for i in range(0,len(ucdp_full)):
    
    ucdp_full["tokens"][i]=word_tokenize(ucdp_full["fulltext"][i])
    ucdp_full["tokens"][i]=re.sub('<[^<]+?>','', ucdp_full["fulltext"][i])
    ucdp_full["tokens"][i]=to_lower(ucdp_full["fulltext"][i])
    ucdp_full["tokens"][i]=remove_num(ucdp_full["fulltext"][i])
    ucdp_full["tokens"][i]=tokenizer.tokenize(ucdp_full["fulltext"][i])



<ipython-input-6-9538196ea661>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ucdp_full["tokens"][i]=word_tokenize(ucdp_full["fulltext"][i])
<ipython-input-6-9538196ea661>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ucdp_full["tokens"][i]=re.sub('<[^<]+?>','', ucdp_full["fulltext"][i])
<ipython-input-6-9538196ea661>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ucdp_full["tokens"][i]=to_lower(ucdp_full["fulltext"][i])
<ipytho

In [13]:
pd.options.display.max_colwidth = 1000000000

ucdp_full[["tokens"]]

,tokens
0,"[Peshawar, Accord, In, the, Name, of, Allah, the, Most, Beneficient, the, Most, Merciful, 22, 10, 1412, 22, Shawal, 24, April, 1992, Salutation, and, peace, be, upon, the, Great, Messenger, of, Allah, and, his, Progeny, and, Companions, The, structure, and, process, for, the, provisional, period, of, t, he, Islamic, State, of, Afghanistan, was, formed, as, under, 1, It, was, decided, that, a, 51, persons, body, headed, by, Hazrat, Sahib, Sibghatullah, Mojaddedi, would, go, inside, Afghanistan, so, that, they, could, take, over, power, from, the, present, rulers, of, Kabul, completely, and, without, any, terms, and, conditions, during, the, two, months, period, The, ...]"
1,"[Afghan, Peace, Accord, Islamabad, Accord, Given, our, submission, to, the, will, of, Allah, Almighty, and, commitment, to, seeking, guidance, from, the, Holy, Quran, and, Sunnah, Recalling, the, glorious, success, of, the, epic, Jehad, waged, by, the, valiant, Afghan, people, against, foreign, occupation, Desirous, of, ensuring, that, the, fruits, of, this, glorious, Jehad, bring, peace, progress, and, prosperity, for, the, Afghan, people, Having, agreed, to, bringing, armed, hostilities, to, an, end, Recognizing, the, need, for, a, broad, based, Islamic, Government, in, which, all, parties, and, groups, representing, all, segments, of, Muslim, Afghan, society, are, represented, so, that, the, process, of, political, ...]"
2,"[Kabul, Agreement, between, the, Government, of, Afghanistan, and, Hizb, i, Islami, yi, Afghanistan, on, 22, September, 2016, Source, Kabul, Times, 26, September, 2016, Full, text, of, agreement, between, GoI, RA, Hezb, e, Islami, of, Afghanistan, led, by, Gulbuddin, Hekmatyar, http, thekabultimes, gov, af, index, php, opinions, politics, 11934, full, text, of, agreement, between, goira, hezb, e, islami, of, afghanistan, led, by, gulbuddin, hekmatyar, html, accessed, 2017, 04, 03, Article, One, With, strong, and, unshakable, faith, to, Allah, the, Almighty, and, sincere, believe, in, holy, religion, of, Islam, both, parties, oblige, themselves, to, unconditional, obedience, of, the, lofty, principles, of, this, ...]"
3,"[The, Gbadolite, Declaration, on, Angola, 1, At, the, invitation, of, Marshal, Mobutu, Sese, Seko, Founder, President, of, the, Popular, Movement, of, the, Revolution, and, President, of, th, e, Republic, of, Zaire, an, historic, meeting, took, place, in, Gbadolite, Zaire, on, Thursday, 22nd, June, 1989, 2, The, following, Heads, of, State, attended, this, important, conference, His, Excellency, Mr, Kenneth, David, Kaunda, Doyen, of, our, region, of, Central, and, Southern, Africa, and, President, of, the, Republic, of, Zambia, Others, in, attendance, were, in, alphabetical, order, by, country, His, Excellency, Mr, José, Eduardo, dos, Santos, President, of, the, Peoples, Republic, of, Angola, His, ...]"
4,"[The, Bicesse, Agreement, The, Government, of, the, People, s, Republic, of, Angola, and, the, National, Union, for, the, Total, Independence, of, Angola, UNITA, with, mediat, ion, by, the, Government, of, Portugal, and, the, participation, of, observers, from, the, Governments, of, the, United, States, of, America, and, the, Union, of, Soviet, Socialist, Republics, Accept, as, binding, the, following, documents, which, constitute, the, Peace, Accords, for, Angola, a, The, cease, fire, agreement, incl, uding, annexes, I, and, II, thereto, b, Fundamental, principles, for, th, e, establishment, of, peace, in, Angola, including, the, annex, thereto, regarding, the, Joint, Political, Military, Commission, c, Concepts, ...]"
...,...
231,"[AGREEMENT, ON, ACCOUNTABILITY, AND, RECONCILIATION, BETWEEN, THE, GOVERNMENT, OF, THE, REPUBLIC, OF, UGANDA, AND, THE, LORD, S, RESISTANCE, ARMY, MOVEMENT, JUBA, SUDAN, This, Agreement, between, the, Government, of, Uganda, The, Government, and, the, Lord, s, Resistance, Army, Movement, LRA, M, herein, refe, rred, to, as, the, Parties, witnesseth, that, PREAMBLE, WHEREAS, THE,

In [ ]:
stop_words=set(stopwords.words("english"))

In [ ]:
for i in range(0,len(ucdp_full)):
    for w in ucdp_full["tokens"][i]:
        if w not in stop_words:
            filtered_sent.append(w)